## Задание 8. — Проекционные методы для краевой задачи

* Реализовать один из проекционных методов: метод Ритца или метод Галеркина.
* Сравнить решения при разных N (например, графически).

## Решение

In [36]:
import numpy as np
from numpy.linalg import det
import pandas as pd
from scipy.special import jacobi
from scipy.misc import derivative
import scipy.integrate as integrate

import plotly.express as px
import plotly.graph_objects as go

Будем решать задание с помощью метода Ритца. 

Реализуем его:

In [37]:
def ritz_method(coeffs_de, bounds, e, de, n):
    '''
        Функция, реализующая метод Ритца

        Parameters
        ----------
        coeffs_de : dict
            словарь c коэффициентами ОДУ
        bounds : list
            граничные условия ОДУ
        e : function
            функция, возврающая значение i-й координатной функции в точке x
        de : function
            функция, возврающая значение производной i-й координатной функции в точке x
        n : int
            количество координатных функций
            
        Returns
        ----------
        res : numpy.array
            решение краевой задачи
    '''
    
    p = coeffs_de['p']
    r = coeffs_de['r']
    f = coeffs_de['f']
    alphas, betas = bounds
    
    # правая часть слау
    F = np.array([integrate.quad(lambda x: f(x)*e(i, x), -1, 1)[0] for i in range(0, n)])
    
    # левая часть слау
    Q_l = 0 if np.isclose(alphas[0], 0) or np.isclose(alphas[1], 0) else (alphas[0] / alphas[1])*p(-1)*(alphas[2] ** 2)
    Q_r = 0 if np.isclose(betas[0], 0)  or np.isclose(betas[1], 0)  else (betas[0] / betas[1])*p(1)*(betas[2] ** 2)
    A = []
    for i in range(0, n):
        A.append([0. for _ in range(0, n)])
        for j in range(n):
            bilin = lambda x: -p(x)*de(i, x)*de(j, x) + r(x)*e(i, x)*e(j, x) + Q_l + Q_r
            A[i][j] = integrate.quad(bilin, -1, 1)[0]
    
    # решаем систему
    res = np.linalg.solve(A, F)
    return res

Зададим функцию тестирования:

In [38]:
def testing(coeffs_de, bounds, e, de, n, count):
    '''
        Функция, тестирующая метод Ритца

        Parameters
        ----------
        coeffs_de : dict
            словарь c коэффициентами ОДУ
        bounds : list
            граничные условия ОДУ
        e : function
            функция, возврающая значение i-й координатной функции в точке x
        de : function
            функция, возврающая значение производной i-й координатной функции в точке x
        n : list
            массив с числами координатных функций
        count: int
            количество точек
    '''
    h = 2 / count
    grid = np.array([i*h - 1 for i in range(0, count + 1)])
    
    fig = go.Figure()
    
    for _n in n:
        sol = ritz_method(coeffs_de, bounds, e, de, _n)
        res = np.array([sum(sol[i]*e(i, x) for i in range(0, _n)) for x in grid])
        
        fig.add_trace(go.Scatter(
            x=grid,
            y=res,
            mode = 'lines',
            name=f'n = {_n}'
        ))

    fig.update_layout(
        showlegend=True,
        width=800,
        height=500,
    )

    fig.show()

Возьмём вариант 2 граничной задачи из методички А.Н.Пакулиной (для метода Ритца):

In [39]:
# функции-коэффициенты
coeffs_de = {
    'p': lambda x: -((2 + x) / (3 + x)),
    'r': lambda x: 1 + np.sin(x),
    'f': lambda x: 1 - x
}

# граничные условия
bounds = [
    [0, 1, 0],
    [1, 1, 0]
]

# координатная система
def e(i, x):
    if i == 0:
        return 1
    elif i == 1:
        return x
    else:
        return (1 - x ** 2)*jacobi(i - 2, 1., 1.)(x)
    
# производные координатной системы    
def de(i, x):
    if i == 0:
        return 0
    elif i == 1:
        return 1
    else:
        return derivative(lambda x: (1 - x ** 2)*jacobi(i, 1., 1.)(x), x)

Сравним графически решения при различных n:

In [40]:
testing(coeffs_de, bounds, e, de, [6, 8, 10, 12], 30)